# Sociedade, Contabilidade e Gestão

## Bibliotecas importadas

In [1]:
import requests
from bs4 import BeautifulSoup

import pandas as pd

## Funções básicas

In [2]:
def criar_query(key_word):
    return "+".join(key_word.split())

In [3]:
def listar_links_pagina(url_search):
    page = requests.get(url_search)
    soup = BeautifulSoup(page.content, 'html.parser')
    tbl_results = soup.find('table', {'class': 'listing'})
    a_links = []
    for a in tbl_results.find_all('a'):
           if 'Abstract' in a.text:
                a_links.append(a['href'])
    return a_links



In [4]:
def listar_links(url_search_text, query, pg_num=1):
    a_links = [] 
    while True:
        url_search = url_search_text.replace('__query__', query).replace('__pg_num__', str(pg_num))
        tmp = listar_links_pagina(url_search)
        if len(tmp) != 0:
            pg_num += 1
            a_links += tmp
        else:
            break
    return a_links



In [5]:
url_search_text = 'https://revistas.ufrj.br/index.php/scg/search/search?query=__query__&searchJournal=55&authors=&title=&abstract=&galleyFullText=&suppFiles=&discipline=&subject=&type=&coverage=&indexTerms=&dateFromMonth=&dateFromDay=&dateFromYear=2012&dateToMonth=&dateToDay=&dateToYear=2021&orderBy=&orderDir=&searchPage=__pg_num__#results'
links = listar_links(url_search_text, 'contabilidade+ambiental', pg_num=1)

link = links[0]

link

'https://revistas.ufrj.br/index.php/scg/article/view/16760'

In [6]:
page = requests.get(link)
soup = BeautifulSoup(page.content, 'html.parser')

In [7]:
def completar_lista(lista):
    if len(lista) < 6:
        lista += (6 - len(lista)) * ['']
    return lista

In [8]:
def captura_titulo(soup):
    try:
        title_tag = soup.find('div', {'id': 'articleTitle'})
        title = title_tag.find('h3').text
    except:
        title = ''
    return {'Título': title}

In [9]:
captura_titulo(soup)['Título']

'Análise da Divulgação de Informações Ambientais por Empresas do Setor Elétrico: um Estudo em Empresas dos Países do BRICS'

In [10]:
def captura_data(soup):
    try:
        year_tags = soup.find('div', {'id': 'breadcrumb'})
        year_tag = year_tags.find_all('a')
        year_text = year_tag[1].text
        year = year_text[-5:-1]
    except:
        year = ''
    return {'Data de Publicação': year}

In [11]:
captura_data(soup)

{'Data de Publicação': '2020'}

In [12]:
def captura_resumo(soup):
    try:
        abstract_tag = soup.find('div', {'id': 'articleAbstract'})
        abstract = abstract_tag.find('div').text
    except:
        abstract = ''
    return {'Resumo': abstract}

In [13]:
captura_resumo(soup)

{'Resumo': 'O objetivo da pesquisa é analisar o nível de divulgação ambiental das empresas dos países do BRICS (Brasil, Rússia, Índia, China e África do Sul). A amostra é composta por\xa0 empresas do setor de energia elétrica, cotadas em bolsa: 30 do Brasil, 10 da Rússia, 25 da Índia, 23 da China e duas da África. O setor elétrico foi escolhido para estudo por ter importância estratégica e estar vinculado a várias das Metas do Milênio da ONU. Os países do BRICS foram escolhidos para estudo devido sua relevância no contexto econômico mundial e seu envolvimento no atingimento do desenvolvimento sustentável. Foram utilizados os indicadores da Global Reporting Initiative como parâmetros de desempenho ambiental. Os países que apresentaram o maior nível de divulgação ambiental foram, nesta ordem, África do Sul, Brasil e Rússia, e aqueles que menos divulgaram foram Índia e China. Se a África do Sul fosse desconsiderada, por contar somente com duas empresas, o Brasil passaria a primeiro lugar 

In [14]:
def captura_palavras_chave(soup):
    try:
        kw_tag = soup.find('div', {'id': 'articleSubject'})
        kw = kw_tag.find('div').text.replace(',', '.').replace(';', '.')
    except:
        kw = ''
    return {'Palavras-chave': kw}

In [15]:
captura_palavras_chave(soup)

{'Palavras-chave': 'Divulgação ambiental. Setor elétrico. BRICS. Relatórios de Sustentabilidade'}

In [16]:
def captura_autores(soup):
    autores = []
    try:
        authors_tag = soup.find('div', {'id': 'authorString'})
        authors_full = authors_tag.find('em')
        autores = authors_full.text.split(', ')
    except:
        pass

    qtd_autores = len(autores)
    autores = completar_lista(autores)
    autores = [qtd_autores] + autores
    
    key_autores = [f"Autor {i}" for i in range(1, 7)]
    key_autores = ['Qtd. de Autores'] + key_autores

    return dict(zip(key_autores, autores))

In [17]:
authors_tag = soup.find('div', {'id': 'authorString'})
authors_full = authors_tag.find('em')
autores = authors_full.text.split(', ')

autores

['Marcelle Colares Oliveira',
 'Francisco Adelmo Medeiros Damasceno',
 'Sandra Michelinne Saraiva de Sousa Amorim',
 'Carlos Adriano Santos Gomes']

In [18]:
captura_autores(soup)

{'Qtd. de Autores': 4,
 'Autor 1': 'Marcelle Colares Oliveira',
 'Autor 2': 'Francisco Adelmo Medeiros Damasceno',
 'Autor 3': 'Sandra Michelinne Saraiva de Sousa Amorim',
 'Autor 4': 'Carlos Adriano Santos Gomes',
 'Autor 5': '',
 'Autor 6': ''}

In [19]:
def captura_afiliacao(soup):
    affiliation_list = []
    try:
        authors_tag = soup.find_all('div', {'class': 'authorBio'})
        for p in authors_tag:
            try:
                affiliation = p.find_all('br')[-2].next_sibling
                affiliation = affiliation.replace('\t', '')
            except:
                affiliation = ''
            affiliation_list.append(affiliation)
    except:
        pass
    affiliation_list = completar_lista(affiliation_list)
    key_affiliation = [f"Afilição {i}" for i in range(1, 7)]

    return dict(zip(key_affiliation, affiliation_list))

In [20]:
captura_afiliacao(soup)

{'Afilição 1': '',
 'Afilição 2': '',
 'Afilição 3': '',
 'Afilição 4': '',
 'Afilição 5': '',
 'Afilição 6': ''}

In [21]:
def realizar_consulta(key_words_list):
    periodico_nome_dict = {'Revista': 'Sociedade, Contabilidade e Gestão'}
    url_search_text = 'https://revistas.ufrj.br/index.php/scg/search/search?query=__query__&searchJournal=55&authors=&title=&abstract=&galleyFullText=&suppFiles=&discipline=&subject=&type=&coverage=&indexTerms=&dateFromMonth=&dateFromDay=&dateFromYear=2012&dateToMonth=&dateToDay=&dateToYear=2021&orderBy=&orderDir=&searchPage=__pg_num__#results'

    a_links = []
    for key_word in key_words_list:
        query = criar_query(key_word)
        a_links += listar_links(url_search_text, query)
        
    resultados = []
    for link in a_links:
        link_dict = {'url': link}
        page = requests.get(link)
        soup = BeautifulSoup(page.content, 'html.parser')
        
        if 'Pré-textual' in captura_titulo(soup)['Título'] or 'CAP Accounting and Management' in captura_titulo(soup)['Título']:
            continue
            
        dict_artigo = {
            **captura_data(soup),
            **captura_titulo(soup),
            **captura_resumo(soup),
            **captura_palavras_chave(soup),
            **captura_autores(soup),
            **captura_afiliacao(soup),
            **link_dict,
            **periodico_nome_dict
        }
        resultados.append(dict_artigo)
    df = pd.DataFrame(resultados)
    df['Data de Publicação'] = pd.to_datetime(df['Data de Publicação'], format='%Y')
    return df

## Pesquisa por artigos

In [22]:
key_words_list = ['contabilidade ambiental', 'balanço social', 'relato integrado', 'nbct 15']
df = realizar_consulta(key_words_list)
df.head()

,Data de Publicação,Título,Resumo,Palavras-chave,Qtd. de Autores,Autor 1,Autor 2,Autor 3,Autor 4,Autor 5,Autor 6,Afilição 1,Afilição 2,Afilição 3,Afilição 4,Afilição 5,Afilição 6,url,Revista
0,2020-01-01,Análise da Divulgação de Informações Ambientai...,O objetivo da pesquisa é analisar o nível de d...,Divulgação ambiental. Setor elétrico. BRICS. R...,4,Marcelle Colares Oliveira,Francisco Adelmo Medeiros Damasceno,Sandra Michelinne Saraiva de Sousa Amorim,Carlos Adriano Santos Gomes,,,,,,,,,https://revistas.ufrj.br/index.php/scg/article...,"Sociedade, Contabilidade e Gestão"
1,2019-01-01,Divulgação Voluntária de Informações Ambientai...,O presente estudo tem por objetivo investigar ...,Disclosure de Informações Ambientais. Sustenta...,4,Yuri Gomes Paiva Azevedo,Vanessa Câmara de Medeiros,Anderson Luiz Rezende Mól,Clayton Levy Lima de Melo,,,,,,,,,https://revistas.ufrj.br/index.php/scg/article...,"Sociedade, Contabilidade e Gestão"
2,2019-01-01,Dialogic Accounting for the Democratic Informa...,"The Federal Government's social program, Bolsa...",Dialogic Accounting. Bolsa Verde. Prestação de...,4,Fátima de Souza Freire,Nilton Oliveira da Silva,Amanda Guimarães Lugon,Erika Rocha Oprea de Carvalho,,,,,,,,,https://revistas.ufrj.br/index.php/scg/article...,"Sociedade, Contabilidade e Gestão"
3,2019-01-01,A Relação entre o Exame de Suficiência Contábi...,O presente estudo objetivou verificar a relaçã...,Exame de Suficiência. Enade. Conceito Prelimin...,2,Paulo Vitor Souza de Souza,Luciana da Silva Moraes Sardeiro,,,,,,,,,,,https://revistas.ufrj.br/index.php/scg/article...,"Sociedade, Contabilidade e Gestão"
4,2011-01-01,Análise do Disclosure Relacionado a Acidentes ...,"Este trabalho tem como objetivo verificar, no ...",,3,Fabiano Helmer Bremenkamp,José Elias Feres de Almeida,Maria Mariete Aragão Melo Pereira,,,,,,,,,,https://revistas.ufrj.br/index.php/scg/article...,"Sociedade, Contabilidade e Gestão"


In [23]:
df.to_csv('../data/SSG_UFRJ.csv', index=False)

In [24]:
df

,Data de Publicação,Título,Resumo,Palavras-chave,Qtd. de Autores,Autor 1,Autor 2,Autor 3,Autor 4,Autor 5,Autor 6,Afilição 1,Afilição 2,Afilição 3,Afilição 4,Afilição 5,Afilição 6,url,Revista
0,2020-01-01,Análise da Divulgação de Informações Ambientai...,O objetivo da pesquisa é analisar o nível de d...,Divulgação ambiental. Setor elétrico. BRICS. R...,4,Marcelle Colares Oliveira,Francisco Adelmo Medeiros Damasceno,Sandra Michelinne Saraiva de Sousa Amorim,Carlos Adriano Santos Gomes,,,,,,,,,https://revistas.ufrj.br/index.php/scg/article...,"Sociedade, Contabilidade e Gestão"
1,2019-01-01,Divulgação Voluntária de Informações Ambientai...,O presente estudo tem por objetivo investigar ...,Disclosure de Informações Ambientais. Sustenta...,4,Yuri Gomes Paiva Azevedo,Vanessa Câmara de Medeiros,Anderson Luiz Rezende Mól,Clayton Levy Lima de Melo,,,,,,,,,https://revistas.ufrj.br/index.php/scg/article...,"Sociedade, Contabilidade e Gestão"
2,2019-01-01,Dialogic Accounting for the Democratic Informa...,"The Federal Government's social program, Bolsa...",Dialogic Accounting. Bolsa Verde. Prestação de...,4,Fátima de Souza Freire,Nilton Oliveira da Silva,Amanda Guimarães Lugon,Erika Rocha Oprea de Carvalho,,,,,,,,,https://revistas.ufrj.br/index.php/scg/article...,"Sociedade, Contabilidade e Gestão"
3,2019-01-01,A Relação entre o Exame de Suficiência Contábi...,O presente estudo objetivou verificar a relaçã...,Exame de Suficiência. Enade. Conceito Prelimin...,2,Paulo Vitor Souza de Souza,Luciana da Silva Moraes Sardeiro,,,,,,,,,,,https://revistas.ufrj.br/index.php/scg/article...,"Sociedade, Contabilidade e Gestão"
4,2011-01-01,Análise do Disclosure Relacionado a Acidentes ...,"Este trabalho tem como objetivo verificar, no ...",,3,Fabiano Helmer Bremenkamp,José Elias Feres de Almeida,Maria Mariete Aragão Melo Pereira,,,,,,,,,,https://revistas.ufrj.br/index.php/scg/article...,"Sociedade, Contabilidade e Gestão"
5,2014-01-01,Contabilidade e Meio Ambiente: Uma Análise da ...,Desde a Antiguidade a atividade econômica vem ...,,1,José Ricardo Maia de Siqueira,,,,,,,,,,,,https://revistas.ufrj.br/index.php/scg/article...,"Sociedade, Contabilidade e Gestão"
6,2016-01-01,Incentivo Fiscal à Cultura: Uma Análise das Di...,A divulgação de informações dos Incentivos Fis...,,4,Luciano Gomes dos Reis,Felipe Pagni,Raissa Gouveia Ancioto,Maria Carolina de Paula Oliveira,,,,,,,,,https://revistas.ufrj.br/index.php/scg/article...,"Sociedade, Contabilidade e Gestão"
7,2011-01-01,Análise do Disclosure Relacionado a Acidentes ...,"Este trabalho tem como objetivo verificar, no ...",,3,Fabiano Helmer Bremenkamp,José Elias Feres de Almeida,Maria Mariete Aragão Melo Pereira,,,,,,,,,,https://revistas.ufrj.br/index.php/scg/article...,"Sociedade, Contabilidade e Gestão"
8,2019-01-01,Pesquisa ou Propaganda? Análise da Literatura ...,O objetivo deste artigo é identificar como os ...,Relato Integrado. Campo científico. Pesquisa c...,1,Paulo Frederico Homero Junior,,,,,,,,,,,,https://revistas.ufrj.br/index.php/scg/article...,"Sociedade, Contabilidade e Gestão"
9,2017-01-01,Relato Integrado: Um estudo da aderência da es...,O Relato Integrado (RI) surgiu a partir da nec...,,2,Betina França Gomes de Freitas,Fátima de Souza Freire,,,,,,,,,,,https://revistas.ufrj.br/index.php/scg/article...,"Sociedade, Contabilidade e Gestão"
